In [4]:
import numpy as np
import pandas as pd
import pickle
import subprocess
import requests

In [5]:
import flask

In [6]:
!pip install uvicorn fastapi

In [7]:
%%file app.py

import pickle
from math import log10
import pandas as pd

from flask import Flask
from flask import request
from flask import jsonify
import numpy as np

import random

class Perceptron():
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        # self.w_ = np.zeros(1+X.shape[1])
        
        self.w_ = [random.uniform(-1.0, 1.0) for _ in range(1+X.shape[1])] 
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X,y):
                #print(xi, target)
                update = self.eta*(target-self.predict(xi))
                #print(update)
                self.w_[1:] += update*xi
                self.w_[0] += update
                #print(self.w_)
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:])+self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X)>=0.0,1,-1)

# Create a flask
app = Flask(__name__)

# Create an API end point
@app.route('/api/v1.0/predict', methods=['GET'])
def get_prediction():

    # sepal length
    sepal_length = float(request.args.get('sl'))
    # sepal width
    #sepal_width = float(request.args.get('sw'))
    # petal length
    petal_length = float(request.args.get('pl'))
    # petal width
    #petal_width = float(request.args.get('pw'))

    # The features of the observation to predict
    #features = [sepal_length,
    #            sepal_width,
    #            petal_length,
    #           petal_width]
    
    features = [sepal_length,
                petal_length]
    
    print(features)
    # Load pickled model file
    with open('model.pkl',"rb") as picklefile:
        model = pickle.load(picklefile)
    print(model)
    # Predict the class using the model
    predicted_class = int(model.predict(features))
    
    # Return a json object containing the features and prediction
    return jsonify(features=features, predicted_class=predicted_class)

if __name__ == '__main__':
    app.run()

Overwriting app.py


In [8]:
%%file requirements.txt
Flask==2.1.0
numpy==1.21.0

Overwriting requirements.txt


In [9]:
%%file Dockerfile

FROM python:3.11.3-slim-buster

WORKDIR /app

COPY app.py .
COPY requirements.txt .

RUN pip install -r requirements.txt

EXPOSE 5000

ENTRYPOINT ["python"]

CMD ["app.py"]

Overwriting Dockerfile


```bash
docker build -t test_hello .

docker run -p 5000:5000 test_hello
```

In [10]:
# http://127.0.0.1:5000/api/v1.0/predict?&sl=4.5&pl=1.3

# pamiętaj otworzyć nowy notebook !
response = requests.get("http://127.0.0.1:5000/api/v1.0/predict?&sl=4.5&pl=1.3")
print(response.content)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=6000): Max retries exceeded with url: /api/v1.0/predict?&sl=4.5&pl=1.3 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CB5CF44190>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))